In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import DataLoader

from torchvision import transforms, datasets

import matplotlib
import matplotlib.pyplot as plt
import time
import argparse

matplotlib.style.use("ggplot")

device = torch.cuda.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ),
    ]
)
val_transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], 
            std=[0.229, 0.224, 0.225]
        ),
    ]
)

In [ ]:
data_root = "../data/isdog"
train_dataset = datasets.ImageFolder(
    root=data_root + "/train",
    transform=train_transform
)

train_dataloader = DataLoader(
    train_dataset, batch_size=32, shuffle=True
)

val_dataset = datasets.ImageFolder(
    root=data_root + "/test",
    transform=val_transform
)

val_dataloader = DataLoader(
    val_dataset, batch_size=32, shuffle=False
)

In [ ]:
def resnet50(pretrained=True, require_grad=False):
    model = models.resnet50(pretrained=pretrained, progress=True)
    if not require_grad:
        for param in model.parameters():
            param.requires_grad = False
    else:
        for param in model.parameters():
            param.requires_grad = True
    model.fc = nn.Linear(2048, 2)
    return model

In [ ]:
class LRScheduler:
    def __init__(self, opti, patience=5, min_lr=1e-6, factor=0.5):
        self.opti = opti
        self.patience = patience
        self.min_lr = min_lr
        self.factor = factor
        self.lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            self.opti, mode="min", patience=self.patience, factor=self.factor, verbose=True
        )
    def __call__(self, val_loss):
        self.lr_scheduler.step(val_loss)

In [ ]:
class EarlyStopping():
    def __init__(self, patience=5, verbose=True, delta=0, path="ckpt/isdog_resnet50.pt"):
        self.patience = patience
        self.counter = 0
        self.verbose = verbose
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        
    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0
            
    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f"Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model ...")
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--lr", desc="lr_scheduler", action="store_true")
parser.add_argument("--es", desc="early_stopping", action="store_true")

args = vars(parser.parse_args())

In [ ]:
print(f"Computation device: {device}")
model = models.resnet50(pretrained=True).to(device)
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params:}")
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters: {total_trainable_params:}")

In [ ]:
lr = 0.001
epochs = 100
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

In [ ]:
loss_plot_name = "loss"

"""
- p.490
"""